In [ ]:
import tensorflow as tf
from keras.layers import Dense, Input, InputLayer, Dropout, BatchNormalization, Convolution2D, MaxPooling2D, GlobalMaxPool2D
from keras import activations, models, optimizers, losses
from keras.activations import relu
from keras.models import Sequential
import pandas as pd
import os

In [ ]:
metadata = pd.read_csv('../UrbanSound8K/metadata/UrbanSound8K.csv')
metadata_fold_one = metadata.query("fold == 1")
metadata_fold_one.sort_values(by='slice_file_name')
metadata_fold_one

# (x_train, y_train) = ((Xaudios, 4000), (Xaudios,))
# (x_test, y_test) = ((Yaudios, 4000), (Yaudios,))

In [ ]:
audio_dataset_path='../UrbanSound8K/audio/'
audio=[]
metadata = pd.read_csv('../UrbanSound8K/metadata/UrbanSound8K.csv')
for index_num,row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    final_class_labels=row["class"]
    
    #data=features_extractor(file_name)
    audio.append([data,final_class_labels])
    
# converting extracted_features to Pandas dataframe
audio_df=pd.DataFrame(audio,columns=['feature','class'])
audio_df.head()

In [ ]:
sum=0
for i in range(10):
    fold = pd.read_csv(f'datasets/fold{i+1}.csv')
    print(f"Length of fold {i+1} is {len(fold)}")
    sum+=len(fold)

print(sum)

In [ ]:
#with open('dataset.yaml') as f:
#    data = yaml.load(f, yaml.loader.BaseLoader)

In [ ]:
mlp = Sequential()
mlp.add(InputLayer(input_shape=(4000, ))) # input layer  #4000 = sample rate 1000 * 4sec audio
mlp.add(Dense(256, activation='relu')) # hidden layer 1
mlp.add(Dense(256, activation='relu')) # hidden layer 2
mlp.add(Dense(10, activation='softmax')) # output layer  #10 = n_class

mlp.compile(loss='categorical_crossentropy',
            metrics=['accuracy'],
            optimizer='adam')
            

# summary
mlp.summary()

In [ ]:
raw_data = pd.read_csv('datasets/fold1.csv')
print(raw_data.shape)

for i in range(872):

In [ ]:
nclass = 10
inp = Input(shape=(157, 320, 1))
norm_inp = BatchNormalization()(inp)
audio = Convolution2D(16, kernel_size=(3, 7), activation=activations.relu)(norm_inp)
audio = Convolution2D(16, kernel_size=(3, 7), activation=activations.relu)(audio)
audio = MaxPooling2D(pool_size=(3, 7))(audio)
audio = Dropout(rate=0.1)(audio)
audio = Convolution2D(32, kernel_size=3, activation=activations.relu)(audio)
audio = Convolution2D(32, kernel_size=3, activation=activations.relu)(audio)
audio = MaxPooling2D(pool_size=(3, 3))(audio)
audio = Dropout(rate=0.1)(audio)
audio = Convolution2D(128, kernel_size=3, activation=activations.relu)(audio)
audio = GlobalMaxPool2D()(audio)
audio = Dropout(rate=0.1)(audio)

dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(audio))
dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(dense_1))
dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

cnn = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam()

cnn.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
cnn.summary()

In [ ]:
print(cnn)